In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# bag of wordsを作成するためのライブラリ
from gensim import corpora, matutils
# TF-IDFを作成するためのライブラリ
from gensim import models

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### サンプルデータの読み込み

In [2]:
#encoding='utf-8'で上手くいった
train_df = pd.read_csv('posi_nega_traindata.csv', encoding='utf-8')

In [3]:
train_df

,text,posi_nega
0,案件個別の技術相談乗ってます！乗ります,1
1,勉強とお仕事が有機的に繋がっていて面白い。,1
2,次々とテクニックを教えていただき、ためになりました,1
3,お肉がキレイに焼けたことに小さなしあわせを感じた,1
4,これが私の最適化,1
5,データの抽出やグラフ化も、問題なさそう。,1
6,PC復帰するまでみんなの日報読んでコメントしてスッキリする。,1
7,久々に新卒が自社にまあまあいて嬉しかった,1
8,おでんが美味しい季節だなぁ,1
9,昼飯には少し割高な値段であったが満足感があった。,1


### neologdn.normalize()を使用して文章全体を正規化しMeCab + neologdで形態素解析、品詞で単語を絞った後に出現頻度0以上の単語を抽出してリストに格納する

In [4]:
def get_mecabed_word_list(text):
    sentence_normalization = neologdn.normalize(text)
    neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')
    
    # neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される
    # 原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
    # 例: '空い\tアイ\t空く\t動詞-自立\t五段・カ行イ音便\t連用タ接続\nた\tタ\tた\t助動詞\t特殊・タ\t基本形\n時間....
    # まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする
    wakati_text_list = neologd_tagger.parse(sentence_normalization).split('\n')
    # 「\n」で区切り、リスト型にした結果の例は下記
    # ['空い\tアイ\t空く\t動詞-自立\t五段・カ行イ音便\t連用タ接続', ：リスト0番目
    # 'た\tタ\tた\t助動詞\t特殊・タ\t基本形',：リスト1番目
    # '時間\tジカン\t時間\t名詞-副詞可能\t\t',：リスト2番目
    
    ##【形態素解析結果を格納したリストから特定の品詞（品詞詳細部分まで考慮に入れた場合）のみ抽出】
    # 抽出したい品詞のリストを作成（完全一致）
    # 品詞参考URL：http://miner.hatenablog.com/entry/323
    hinshi_list = ['名詞-一般', '名詞-形容動詞語幹', '名詞-固有名詞-一般',  '名詞-サ変接続', '形容詞-自立', '形容詞-接尾', '形容詞-非自立', '動詞-自立', '動詞-接尾', '動詞-非自立', '副詞-一般', '副詞-助詞類接続']
    # hinshi_list = ('名詞-一般', '名詞-サ変接続', '名詞-固有名詞', '名詞-形容動詞語幹'...)とタプルでも同じ結果
    original_form_list =[]# 単語の「原形」のみ格納する(品詞情報は単語頻度を求める際に必要ないので除外)
    
    # parse() の出力結果の最後は「EOS」という文字のみ
    # EOSのとき、pos = wakati.split('\t')[3]の要素はないので下記forループを実行すると「list index out of range」とエラーを発生させてしまう
    # よってEOSのときは条件分岐if~breakでforループから抜け出すよう記述
    for wakati in wakati_text_list:
        surface = wakati.split('\t')[0]
        if surface == 'EOS':
            break
        else:
            pos = wakati.split('\t')[3]
            if pos in hinshi_list:# posはhinshi_listの中の要素と完全一致していないと抽出できない
                original_form_list.append(wakati.split('\t')[2])
    # ここまでで品詞によって単語を絞った
    
    ##【単語の出現頻度を求め、出現頻度0以上の単語のみ抽出】
    # まずは単語の出現頻度を求める
    import collections
    count = collections.Counter(original_form_list)
    word_count_list = count.most_common()
    word_count_list# word_count_listは全体がリスト型でリスト内に('する', 4), ('ぼんやり', 3),,,,といった単語とその出現頻度がタプルで格納されている
    # 次に出現頻度が0以上（今回はテキストデータが少ないので）の単語のみ抽出する
    Frequency1_word_list = []
    for word_count in word_count_list:
        if word_count[1] > 0:
            Frequency1_word_list.append(word_count[0])
    
    return Frequency1_word_list

### 素性ベクトル（文章の特徴を表現したベクトル）への変換

In [5]:
# train_dfのtextカラムのすべての文章に「get_mecabed_word_list」関数（形態素解析する関数）を適応させたい

# apply関数の引数に関数を引き渡すことが可能、関数を全列に適応できる
#  参考URL：https://teratail.com/questions/134846
# train_dfに関数を全列新たなカラム['Wakati']を追加する
# 参考URL：https://datumstudio.jp/blog/%e3%80%90%e7%89%b9%e5%88%a5%e9%80%a3%e8%bc%89%e3%80%91-%e3%81%95%e3%81%81%e3%80%81%e8%87%aa%e7%84%b6%e8%a8%80%e8%aa%9e%e5%87%a6%e7%90%86%e3%82%92%e5%a7%8b%e3%82%81%e3%82%88%e3%81%86%ef%bc%81-3
train_df['Wakati'] = train_df['text'].apply(get_mecabed_word_list)

In [6]:
train_df

,text,posi_nega,Wakati
0,案件個別の技術相談乗ってます！乗ります,1,"[乗る, 案件, 個別, 技術, 相談]"
1,勉強とお仕事が有機的に繋がっていて面白い。,1,"[勉強, お仕事, 有機的, 繋がる, いる, 面白い]"
2,次々とテクニックを教えていただき、ためになりました,1,"[次々, テクニック, 教える, いただく, なる]"
3,お肉がキレイに焼けたことに小さなしあわせを感じた,1,"[お肉。, キレイ, 焼ける, しあわせ, 感じる]"
4,これが私の最適化,1,[最適化]
5,データの抽出やグラフ化も、問題なさそう。,1,"[データ, 抽出, グラフ]"
6,PC復帰するまでみんなの日報読んでコメントしてスッキリする。,1,"[する, PC, 復帰, 日報, 読む, コメント, スッキリ]"
7,久々に新卒が自社にまあまあいて嬉しかった,1,"[まあ, 久々, 新卒, いる, 嬉しい]"
8,おでんが美味しい季節だなぁ,1,"[おでん, 美味しい, 季節]"
9,昼飯には少し割高な値段であったが満足感があった。,1,"[昼飯, 少し, 割高, 値段, 満足感, ある]"


In [7]:
# Wakatiカラムの1行毎に格納されている単語はリストに格納されているのでリスト型と判断されCountVectorizer.fit_transform()が使用できない
# そこでjoin関数でリストの中身を文字列に変換する
# ただし、リストに格納されてる要素に数値が混ざってる場合はmap([適用関数], [対象リスト])を使う必要あり
# 参考URL：https://www.okadalabo.com/python%E3%83%AA%E3%82%B9%E3%83%88%E3%81%AE%E4%B8%AD%E8%BA%AB%E3%82%92%E6%96%87%E5%AD%97%E5%88%97%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B%E3%80%82/
'''
for i in range(0, len(train_df)):
    train_df['Wakati'][i] = ','.join((map(str, train_df['Wakati'][i] )))
# 上記コードを実行すると「 SettingWithCopyWarning」という警告が表示される
# 理由はtrain_df['Wakati'][i]で、まずtrain_df['Wakati']列を取得し、次にtrain_df['Wakati']列のindex0番目の要素にアクセスするという2段階を踏んでいるため
# すなわち、DataFrameを行列とみなした場合、行列の要素に直接アクセスしているのではなく、一度列ベクトルを作って、そして、その列ベクトルの所定の行にアクセスしているのである。
# もし、行数が多い場合は、行数分のメモリの確保とコピーに時間がかかる問題がある。
# 参考URL：http://hirotaka-hachiya.hatenablog.com/entry/2016/01/04/171349
'''
# このSettingWithCopyWarningを回避するためには、.loc[row_indexer,col_indexer]を使えば良い。
# つまり、DataFrameを行列と見なした場合に、一般的な行列表現の（'行'、'列')要素にアクセスするのが、.loc['行','列']で書けることから直感的に記述でき汎用性があると思われる
# 参考URL：http://hirotaka-hachiya.hatenablog.com/entry/2016/01/04/171349
for i in range(0, len(train_df)):
    train_df.loc[i, 'Wakati'] = ','.join((map(str, train_df.loc[i, 'Wakati'])))

In [8]:
train_df

,text,posi_nega,Wakati
0,案件個別の技術相談乗ってます！乗ります,1,"乗る,案件,個別,技術,相談"
1,勉強とお仕事が有機的に繋がっていて面白い。,1,"勉強,お仕事,有機的,繋がる,いる,面白い"
2,次々とテクニックを教えていただき、ためになりました,1,"次々,テクニック,教える,いただく,なる"
3,お肉がキレイに焼けたことに小さなしあわせを感じた,1,"お肉。,キレイ,焼ける,しあわせ,感じる"
4,これが私の最適化,1,最適化
5,データの抽出やグラフ化も、問題なさそう。,1,"データ,抽出,グラフ"
6,PC復帰するまでみんなの日報読んでコメントしてスッキリする。,1,"する,PC,復帰,日報,読む,コメント,スッキリ"
7,久々に新卒が自社にまあまあいて嬉しかった,1,"まあ,久々,新卒,いる,嬉しい"
8,おでんが美味しい季節だなぁ,1,"おでん,美味しい,季節"
9,昼飯には少し割高な値段であったが満足感があった。,1,"昼飯,少し,割高,値段,満足感,ある"


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
# CountVectorizerを利用して単語と列番号の対応付けを実行 Document-Term Matrixを獲得できる
feature_vectors = CV.fit_transform(train_df['Wakati'])
# どの単語を学習しているのかをCV.get_feature_names()で確認
vocabulary = CV.get_feature_names()

In [10]:
print(feature_vectors)

  (0, 148)	1
  (0, 120)	1
  (0, 75)	1
  (0, 135)	1
  (0, 68)	1
  (1, 183)	1
  (1, 10)	1
  (1, 157)	1
  (1, 131)	1
  (1, 15)	1
  (1, 89)	1
  (2, 36)	1
  (2, 8)	1
  (2, 124)	1
  (2, 53)	1
  (2, 138)	1
  (3, 118)	1
  (3, 22)	1
  (3, 143)	1
  (3, 47)	1
  (3, 16)	1
  (4, 130)	1
  (5, 48)	1
  (5, 121)	1
  (5, 54)	1
  :	:
  (36, 145)	1
  (36, 104)	1
  (36, 112)	1
  (36, 28)	1
  (37, 24)	1
  (37, 11)	1
  (37, 4)	1
  (37, 184)	1
  (37, 57)	1
  (37, 29)	1
  (37, 30)	1
  (37, 36)	1
  (38, 113)	1
  (38, 96)	1
  (38, 123)	1
  (38, 78)	1
  (38, 35)	1
  (38, 28)	1
  (38, 10)	1
  (39, 38)	1
  (39, 116)	1
  (39, 122)	1
  (39, 179)	1
  (39, 83)	1
  (39, 10)	1


In [11]:
print('今回の文章で学習した単語の数は{}語です。'.format(len(CV.get_feature_names())))
# print('今回の文章で学習した単語の数は{}語です。'.format(len(CV.vocabulary_.keys()))) 上記はこの表現でも同じ
print('--------------------------------------------------------------------------------------------')
print(vocabulary)# vocabulary = CV.get_feature_names()
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_)
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_.keys())
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_.values())

今回の文章で学習した単語の数は185語です。
--------------------------------------------------------------------------------------------
['10歳', '2週', 'pc', 'ある', 'あんまり', 'いい', 'いう', 'いける', 'いただく', 'いふ', 'いる', 'おいしい', 'おかげ', 'おでん', 'おもしろい', 'お仕事', 'お肉', 'かなり', 'きちんと', 'くる', 'くれる', 'ことごとく', 'しあわせ', 'しまう', 'しょんぼり', 'しれる', 'すぎる', 'すごい', 'する', 'ずっと', 'てる', 'できる', 'でる', 'とても', 'とる', 'どう', 'なる', 'ひとえに', 'ほとんど', 'まあ', 'まとめる', 'もっと', 'やり取り', 'よい', 'れる', 'イレギュラー', 'オフィス', 'キレイ', 'グラフ', 'コメント', 'コード', 'サービス', 'スッキリ', 'テクニック', 'データ', 'ドローン', 'パワポ', 'パン', 'ビジネスモデル', 'ベイズ', 'マジで', 'ミス', 'メンタル', 'モデリング', 'モデル', '上長', '不明瞭', '久々', '乗る', '人間', '会社', '低い', '体調', '作り方', '作業', '個別', '値段', '元気', '先方', '全然', '内容', '出る', '出展', '出来る', '出社', '初めて', '判定', '割高', '助ける', '勉強', '協力', '単純', '原因', '参加', '反省', '取り上げる', '報告', '増える', '変わる', '変化', '変数', '多い', '大丈夫', '大人', '大元', '嬉しい', '季節', '宣伝', '対象', '少し', '常駐', '復帰', '思う', '悩む', '悪い', '悲しい', '情報共有', '意図', '感じる', '打ち合わせ', '技術', '抽出', '担当', '担当者', '教える', '教わる', '整理', '新卒', '日報', '昼飯', '最適化

In [12]:
print(feature_vectors.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
# 疎行列をデータに、インデックスを元のテキストに、カラムを「文章から抽出した単語」したデータフレームを作成
# 書籍「Pythonによるあたらしいデータ分析の教科書 P.290」が参考になるかも
# columns=CV.vocabulary_.keys()にすると単語の並びとfeature_vectors.toarray()の0,1の並びとが対応できていないのでcolumnsにはCV.get_feature_names()を指定する
# 後にtrain_dfの[posi_nega]列とbow_dfをindex をキーとして結合したいのでreset_index()を追加
bow_df = pd.DataFrame(feature_vectors.toarray(), index=train_df['text'], columns=vocabulary ).reset_index()
#  index をキーとして結合したい場合は、df.join()が便利
# 参考URL：http://sinhrks.hatenablog.com/entry/2015/01/28/073327
bow_df2 = bow_df.join(train_df['posi_nega'])

In [14]:
bow_df

,text,10歳,2週,pc,ある,あんまり,いい,いう,いける,いただく,...,軽い,辛い,迷惑,連続,部署,間に合う,難しい,静か,面白い,食べる
0,案件個別の技術相談乗ってます！乗ります,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,勉強とお仕事が有機的に繋がっていて面白い。,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,次々とテクニックを教えていただき、ためになりました,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,お肉がキレイに焼けたことに小さなしあわせを感じた,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,これが私の最適化,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,データの抽出やグラフ化も、問題なさそう。,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,PC復帰するまでみんなの日報読んでコメントしてスッキリする。,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,久々に新卒が自社にまあまあいて嬉しかった,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,おでんが美味しい季節だなぁ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,昼飯には少し割高な値段であったが満足感があった。,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# [posi_nega]列が結合されているか確認
bow_df2

,text,10歳,2週,pc,ある,あんまり,いい,いう,いける,いただく,...,辛い,迷惑,連続,部署,間に合う,難しい,静か,面白い,食べる,posi_nega
0,案件個別の技術相談乗ってます！乗ります,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,勉強とお仕事が有機的に繋がっていて面白い。,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,次々とテクニックを教えていただき、ためになりました,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,お肉がキレイに焼けたことに小さなしあわせを感じた,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,これが私の最適化,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,データの抽出やグラフ化も、問題なさそう。,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,PC復帰するまでみんなの日報読んでコメントしてスッキリする。,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,久々に新卒が自社にまあまあいて嬉しかった,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,おでんが美味しい季節だなぁ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,昼飯には少し割高な値段であったが満足感があった。,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
# to_csv関数でindexにカラム名を名付けるには「index_label='カラム名'」 を引数に追加する
#bow_df2.to_csv('posinega_bow_df.csv', index_label='text', encoding='shift-jis')
# 今回はindex番号を付けたので「index_label='text'」とする必要はなく、「index=False」を引数に与えてindexを出力しないよう設定する
bow_df2.to_csv('posinega_bow_df.csv', index=False, encoding='shift-jis')